In [1]:
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, BatchNormalization, Dropout

import numpy as np

2022-12-01 22:04:49.798602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
JSON_PATH = "./data/stft_data_16sec.json"

with open(JSON_PATH, "r") as fp:
    data_l = json.load(fp)

In [4]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X = np.array(data_l["stft"])
    y = np.array(data_l["labels"])

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [5]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_datasets(0.25, 0.2)

In [6]:
X_train.shape, X_val.shape, X_test.shape

((84, 257, 2757, 1), (21, 257, 2757, 1), (35, 257, 2757, 1))

In [7]:
input_shape = X_train.shape[1:]
input_shape

(257, 2757, 1)

In [36]:
NUM_LABELS = 4

model = Sequential()

# 1st conv layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(BatchNormalization())

# 2nd conv layer
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(BatchNormalization())

# 3rd conv layer
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model.add(BatchNormalization())

# flatten output and feed it into dense layer
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(NUM_LABELS, activation='softmax'))

In [37]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 255, 2755, 32)     320       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 128, 1378, 32)    0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 128, 1378, 32)    128       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 126, 1376, 32)     9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 63, 688, 32)      0         
 g2D)                                                            
                                                      

In [38]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=20)

Epoch 1/20
3/3 [==============================] - 39s 9s/step - loss: 5.0330 - accuracy: 0.2857 - val_loss: 86.1350 - val_accuracy: 0.3810
Epoch 2/20
3/3 [==============================] - 28s 9s/step - loss: 1.8165 - accuracy: 0.7143 - val_loss: 128.6978 - val_accuracy: 0.3810
Epoch 3/20
3/3 [==============================] - 32s 11s/step - loss: 1.0179 - accuracy: 0.7500 - val_loss: 117.4970 - val_accuracy: 0.3810
Epoch 4/20
3/3 [==============================] - 38s 13s/step - loss: 0.7465 - accuracy: 0.7857 - val_loss: 106.9606 - val_accuracy: 0.3810
Epoch 5/20
3/3 [==============================] - 29s 9s/step - loss: 0.5315 - accuracy: 0.8214 - val_loss: 95.7362 - val_accuracy: 0.3810
Epoch 6/20
3/3 [==============================] - 42s 10s/step - loss: 0.3109 - accuracy: 0.8929 - val_loss: 88.3377 - val_accuracy: 0.3810
Epoch 7/20
3/3 [==============================] - 41s 9s/step - loss: 0.3637 - accuracy: 0.8571 - val_loss: 83.0050 - val_accuracy: 0.3810
Epoch 8/20
3/3 [=====

In [39]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

2/2 - 2s - loss: 52.8144 - accuracy: 0.2286 - 2s/epoch - 1s/step
